# Naive Bayes Classifier

For this notebook we are going to yet again use the Titanic data we used in previous notebooks:

| ID | name | survived | sex | age group | PcClass | SibSp |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|1 | Braund, Mr. Owen Harris | 0 | male | adult | 3 | Y |
|2 | Cumings, Mrs. John Bradley | 1 | female | adult | 1 | Y |
|8 | Palsson, Master. Gosta Leonard | 0 | male | child | 3 | Y |
|11 | Sandstrom, Miss. Marguerite Rut | 1 | female | child | 1| Y |
| 40 | Nicola-Yarred, Miss. Jamila | 1 | female | teen | 1 | Y |
|50 | Arnold-Franchi, Mrs. Josef | 0 | female | teen | 3 | Y |
|205 | Cohen, Mr. Gurshon ""Gus" | 1 | male | teen |3 | N |
|307 | Allison, Master. Hudson Trevor | 1 | male | child | 1 | Y |
|507| Quick, Mrs. Frederick Charles | 1 | female | adult | 2 | Y |
|529 | Salonen, Mr. Johan Werner | 0 | male | adult | 3 | N |
|581 | Christy, Miss. Julie Rachel | 1 | female | adult | 2 | Y |
| 871 | Balkic, Mr. Cerin | 0 | male | adult | 3 | N |
| 17 | Rice, Master. Eugene | 0 | male | child | 3 | Y |
|16 |Hewlett, Mrs. (Mary D Kingcome)| 1 | female | adult | 2 | N|
|68 | Crease, Mr. Ernest James | 0 | male | teen | 3 | N |
|263| Taussig, Mr. Emil | 0 | male | adult | 1 | N |
|330| Hippach, Miss. Jean Gertrude | 1 | female | teen | 1 | N|
|505 | Maioni, Miss. Roberta | 1 | female | teen| 1| N |
|533 | Elias, Mr. Joseph Jr | 0 | male | teen | 3 | Y |
| 731 | Ilmakangas, Miss. Pieta Sofia | 0 | female | adult | 3 | Y |


Here is a function that reads this information from a file.



In [5]:
from urllib.request import urlopen 
def readDataFile(urlFile, dataFormat, skipHeader=True):
    
    html = urlopen(urlFile)    
    tempLines = html.read().decode('UTF-8').split('\n')
    #header = lines[0].strip().split('\t')
    
    dFormat = dataFormat.split("\t")
    attributes = []
    for i in range(len(dFormat)):
        if dFormat[i] == 'class':
            classificationColumn = i
            
        elif dFormat[i] == 'attr':
            attributes.append(i)
    #print(classificationColumn, attributes)
    
    
    data = []
    if skipHeader == True:
        lines = tempLines[1:]
    else:
        lines = tempLines
    for line in lines:
        attrVector = []
        instance = line.strip().split('\t')
        #currentClass = "%s=%s" %(classificationName, instance[classificationColumn])
        currentClass = instance[classificationColumn]
        for attribute in attributes:
            attrVector.append(instance[attribute])
        
        
        data.append((currentClass, attrVector))
    return data

    
theFormat = 'comment\tcomment\tclass\tattr\tattr\tattr\tattr'
data = readDataFile('https://raw.githubusercontent.com/zacharski/cs419/master/entropyNotebookData.txt', theFormat) 
print(data)

[('0', ['male', 'adult', '3', 'Y']), ('1', ['female', 'adult', '1', 'Y']), ('0', ['male', 'child', '3', 'Y']), ('1', ['female', 'child', '1', 'Y']), ('1', ['female', 'teen', '1', 'Y']), ('0', ['female', 'teen', '3', 'Y']), ('1', ['male', 'teen', '3', 'N']), ('1', ['male', 'child', '1', 'Y']), ('1', ['female', 'adult', '2', 'Y']), ('0', ['male', 'adult', '3', 'N']), ('1', ['female', 'adult', '2', 'Y']), ('0', ['male', 'adult', '3', 'N']), ('0', ['male', 'child', '3', 'Y']), ('1', ['female', 'adult', '2', 'N']), ('0', ['male', 'teen', '3', 'N']), ('0', ['male', 'adult', '1', 'N']), ('1', ['female', 'teen', '1', 'N']), ('1', ['female', 'teen', '1', 'N']), ('0', ['male', 'teen', '3', 'Y']), ('0', ['female', 'adult', '3', 'Y'])]


As you can see, each line of the data file is converted to a tuple. The first element of the tuple is the classification of that instance, and the second element is a list of the attribute values of that instance.

So this row of the table

| ID | name | survived | sex | age group | PcClass | SibSp |
| :--: | :--: | :--: | :--: | :--: | :--: | :--: |
|1 | Braund, Mr. Owen Harris | 0 | male | adult | 3 | Y |

is converted to the tuple

`('0', ['male', 'adult', '3', 'Y'])`


## Prior probability
The first thing we need to do is compute the prior probability (in our case `P(survived)` and `P(not survived)`). Let's call this function `computePrior` and the function signature will be

`computePrior(data)`

which will compute the prior probabilities of the classifications. In this example, the classifications are '1' meaning *survived* and '0' meaning *not survived.* It should return a dictionary of the form:

`{'0': 0.5, '1': 0.5}`

So `P(survived=0) = 0.5` and `P(survived=1) = 0.5`


####Checkpoint 1: 10xp

Go ahead and write & test that function.


In [31]:
from collections import defaultdict

def computePrior(data):
    ### To Be Done (TBD)
    results = defaultdict(int)
    # TBD
    return results

priorProbabilities = computePrior(data)
print(priorProbabilities)

defaultdict(<class 'int'>, {'0': 0.5, '1': 0.5})


my answer: `defaultdict(<class 'int'>, {'0': 0.5, '1': 0.5})`

##Conditional Probabilities

Next, we need to compute conditional probabilities, such as `P(PcClass=1|survived = 1)`

Let's call this `computeConditionals` and the signature will be

`computeConditionals(data)`

For each of the attribute columns, we are computing all the conditional probabilities of the form `P(attributeColumn=x | classificationColumn=y)

The `classificationColumn` will be the name of the column that contains the categories we are trying to train our classifier to predict (in this case `survived`). 

The output will be a list of conditional probabilities. The data structure to store the probabilities is up to you. I used the following:

`defaultdict(lambda : defaultdict(lambda : defaultdict(int)))`

So a dictionary of dictionaries of dictionaries of integers. 

The probability `P(column0=male|survived=1)` is represented as `result['1'][0]['male']`

where '1' is the value of the survived column, 0 represents the 0th column and 'male' is the value of that 0th column.

Once I collect the counts, I need to convert them to probabilities by iterating over the counts. So something like:

    # now compute probability
    for (category, columns) in results.items():
       for (column, values) in columns.items():
           for (value, count) in values.items():
               # do work here
               



The format of the data is:

    [('0', ['male', 'adult', '3', 'Y']), 
     ('1', ['female', 'adult', '1', 'Y']), 
     ('0', ['male', 'child', '3', 'Y']), 
     ('1', ['female', 'child', '1', 'Y']), 
     ('1', ['female', 'teen', '1', 'Y']), 
     ('0', ['female', 'teen', '3', 'Y']), 
     ('1', ['male', 'teen', '3', 'N']), 
     ('1', ['male', 'child', '1', 'Y']), 
     ('1', ['female', 'adult', '2', 'Y']), 
     ...]
     
and, again, we are going to compute probabilities like `P('male'|'1')` (the probability of a male given that the person survived).

Before you write this function, you should know what you expect the function to return (how will you know that the function is implemented correctly. To that end please compute by hand the following (edit this cell and execute it):

#### Checkpoint 2 10 xp

| probability | value |
| :---: | ---: |
| P(male &#124; survived) | 0.25 |
|P(1st class &#124; survived) | ? |
|P(2nd class &#124; survived) | ? |
|P(3rd class &#124; survived) | ? |
|P(adult &#124; survived) | ? |

In [8]:
from collections import defaultdict
def computeConditionals(data):
    categoryCounts = defaultdict(int)
    results = defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
    # TBD
    return results

conditionalProbabilities = computeConditionals(data)
print(conditionalProbabilities)

print(conditionalProbabilities['1'][0]['male']) # should return .25
print(conditionalProbabilities['1'][1]['teen']) # should return .3846...
## please add test cases

defaultdict(<function computeConditionals.<locals>.<lambda> at 0x105a26f28>, {})
0
0


that was

####Checkpoint 3: 40xp

My output is 

    defaultdict(<class 'int'>, {'0': 10, '1': 10})
    [['male', 'female'], ['adult', 'child', 'teen'], ['3', '1', '2'], ['Y', 'N']]
    defaultdict(<function computeConditionals.<locals>.<lambda> at 0x108e0d488>, {'0': defaultdict(<function         
    computeConditionals.<locals>.<lambda>.<locals>.<lambda> at 0x108e0d378>, {0: defaultdict(<class 'int'>, {'male': 
    0.8, 'female': 0.2}), 1: defaultdict(<class 'int'>, {'child': 0.2, 'teen': 0.3, 'adult': 0.5}), 2: 
    defaultdict(<class 'int'>, {'1': 0.1, '3': 0.9}), 3: defaultdict(<class 'int'>, {'Y': 0.6, 'N': 0.4})}), '1': 
    defaultdict(<function computeConditionals.<locals>.<lambda>.<locals>.<lambda> at 0x108e0d950>, {0: 
    defaultdict(<class 'int'>, {'male': 0.2, 'female': 0.8}), 1: defaultdict(<class 'int'>, {'child': 0.2, 'teen': 
    0.4, 'adult': 0.4}), 2: defaultdict(<class 'int'>, {'1': 0.6, '3': 0.1, '2': 0.3}), 3: defaultdict(<class 'int'>, 
    {'Y': 0.6, 'N': 0.4})})})

    0.25

    0.38461538461538464




### Classifier

Finally we are going to use the prior and conditional probabilities to classify instances. For example, did Miss Anna McGowan, a teen in third class with no siblings onboard survive or not? We would represent those attributes in the vector `['female', 'teen', '1', 'Y']` and we might call our classifier with something like:

`classify(['female', 'teen', '1', 'Y'], priorProbabilities, conditionalProbabilities)`

To help in verifying your code is correct I would like you to output a tuple of the form:

`('1', 0.9292035398230087)`

meaning we would classify Miss McGowan as '1' (survived) with the normalized probability of .9292


Go ahead and write that function now:

In [14]:
from operator import itemgetter

def classifier(instance, priorP, conditionalP):
    results = []
    # just adding these 2 lines to have something
    t = 1
    results.append(('temp', 9999))
    r2 =  sorted(results, key=itemgetter(1), reverse=True)
    return((r2[0][0], r2[0][1] / t))

classifier(['female', 'teen', '1', 'Y'], priorProbabilities, conditionalProbabilities)

('temp', 9999.0)

#### my output was

`('1', 0.9292035398230087)`

that was

#### Checkpoint 4 - 40xp

#####Congratulations you just finished writing a Bayes classifier!

# Additional work:
### 75 xp for the first of these 40 xp for the other.

####Can you write a classifier that trains on the full Titanic dataset? 
You will need to convert the column age to be categories like 'teen', 'adult', 'child'
Test it on 100 instances you haven't trained on and print out the accuracy.

####Can you write a classifier that works with [The House Vote dataset](https://raw.githubusercontent.com/zacharski/pg2dm-python/master/data/ch6/house-votes.zip)?

### remember a decision tree program that uses the full Titanic dataset is worth 100xp
